## CNN with BN in TensorFlow

* DCGAN 의 CNN 모델을 만들어서 MNIST classification 을 수행해보자. 
* TF 에서 BN 을 적용하는 걸 연습하는 용도. 
* MNIST 로 정확도를 테스트하기 어렵다면 다른 데이터셋도 구해서 적용해보자.

Discriminator of DCGAN:

![Discriminator of DCGAN](http://bamos.github.io/data/2016-08-09/discrim-architecture.png)

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
def weight_init(shape):
    return tf.truncated_normal(shape, stddev=0.1)

In [ ]:
# 일단 MNIST datset 은 28x28x1 이므로, 
X = tf.placeholder(tf.float32, shape=[None, 784])
Y = tf.placeholder(tf.float32, shape=[None, 10])
W1 = tf.Variable(weight_init([]))